In [ ]:
import geopandas as gpd
import pandas as pd
import mytools

In [ ]:
print('标准的列名如下：请按照标准事先写好列名后，再导入数据\n CGI , 地市 , 区县 , 厂家 , 移动总采样点 , 移动弱覆盖采样点 , \
联通总采样点 , 联通弱覆盖采样点 , 电信总采样点 , 电信弱覆盖采样点 , 是否核心城区 , 移动覆盖率')

In [ ]:
data = pd.read_csv('g:/1-规划/表格类保存/2020年覆盖问题进展概况/基础表格/覆盖指标统计-20年03月.csv',encoding='gbk')

In [ ]:
# data = data.loc[data['地市']=='武汉']

In [ ]:
data['cmcc'] = 1- data['移动弱覆盖采样点'] / data['移动总采样点']
data['un'] = 1- data['联通弱覆盖采样点'] / data['联通总采样点']
data['te'] = 1- data['电信弱覆盖采样点'] / data['电信总采样点']

In [ ]:
data_row = data.loc[data['移动总采样点']>0].shape[0]

In [ ]:
cmcc_fgl = 1- data['移动弱覆盖采样点'].sum() / data['移动总采样点'].sum()

In [ ]:
data_atu = data.loc[data['是否核心城区']=='是']

In [ ]:
cmcc_atu_fgl = 1- data_atu['移动弱覆盖采样点'].sum() / data_atu['移动总采样点'].sum()

In [ ]:
data_weak = data[(data['cmcc'] <0.9)&(data['移动弱覆盖采样点']>0)].agg({'CGI':'count'})[0]

In [ ]:
data.loc[data['联通总采样点']>1000,'un_valid']='是'
data.loc[data['电信总采样点']>1000,'te_valid']='是'

In [ ]:
def hd(data,cmcc='',jingdui='',name='un_hd',jd_valid=''): #黑点打标
    
    data[jingdui]=pd.to_numeric(data[jingdui])
    data[cmcc]=pd.to_numeric(data[cmcc])
    data.loc[(((data[cmcc]<0.8)&(data[jingdui]>=0.8)) | ((data[cmcc]>=0.8)&(((data[jingdui])-(data[cmcc]))>0.05)))&
             (data[jd_valid]=='是'),name]='黑点'
    return data

In [ ]:
data=hd(data,'cmcc','te','te_hd','te_valid')
data=hd(data,'cmcc','un','un_hd','un_valid')

In [ ]:
# un_hd = data.loc[data['un_hd']=='黑点'].agg({'CGI':'count'})[0]
# un_valid = data.loc[(data['un_valid']=='是')].agg({'CGI':'count'})[0]
# un_hd_v = un_hd / data_row

In [ ]:
# te_hd = data.loc[data['te_hd']=='黑点'].agg({'CGI':'count'})[0]
# te_valid = data.loc[(data['te_valid']=='是')].agg({'CGI':'count'})[0]
# te_hd_v = te_hd / data_row

In [ ]:
data.loc[(data['te_hd']=='黑点')].agg({'CGI':'count'})[0]

In [ ]:
data.loc[(data['te_hd'] =='黑点')|(data['un_hd'] =='黑点'),'hd']='是'
hd_use = data.loc[data['hd']=='是'].agg({'CGI':'count'})[0]
problem_cell = data.loc[(data['cmcc'] <0.9)|(data['hd'] =='是')].agg({'CGI':'count'})[0]
print('黑点共计{}个，小区共计{}个，劣于竞对小区占比{:.2%},总的问题小区为{}个。'.format(
    hd_use,data_row,hd_use/data_row,problem_cell))

In [ ]:
# if un_hd>te_hd:
#     hd_use = un_hd
#     hd_valid_use = un_valid
#     hd_v_use = un_hd_v
#     problem_cell = data.loc[(data['cmcc'] <0.9)&(data['un_hd'] =='黑点')].agg({'CGI':'count'})[0]
#     print('劣于竞对小区占比是联通，黑点（联通）{}个，有效小区{}个，劣于竞对小区占比{:.2%},总的问题小区为{}个。'.format(
#         hd_use,hd_valid_use,hd_v_use,problem_cell))
# else:
#     hd_use = te_hd
#     hd_valid_use = te_valid
#     hd_v_use = te_hd_v
#     problem_cell = data.loc[(data['cmcc'] <0.9)&(data['te_hd'] =='黑点')].agg({'CGI':'count'})[0]
#     print('劣于竞对小区占比是电信，黑点（电信）{}个，有效小区{}个，劣于竞对小区占比{:.2%},总的问题小区为{}个。'.format(
#         hd_use,hd_valid_use,hd_v_use,problem_cell))

In [ ]:
def cmcc_srtong(data,cmcc='',un='',te='',name='cmcc_strong'): #黑点打标
    data[te]=pd.to_numeric(data[te])
    data[un]=pd.to_numeric(data[un])
    data[cmcc]=pd.to_numeric(data[cmcc])
    data.loc[(((data[cmcc]>=0.8)&((data[un]<0.8)&(data[te]<0.8))) |
             ((data[cmcc]>=0.8)&(((((data[cmcc])-(data[un]))>=0.05))&((((data[cmcc])-(data[te]))>=0.05)))))
             &((data['un_valid']=='是')&(data['te_valid']=='是')),name]='强'
    return data

In [ ]:
data=cmcc_srtong(data,'cmcc','te','un')

In [ ]:
cmcc_srtong_son = data.loc[(data['cmcc_strong']=='强')&((data['un_valid']=='是')&(data['te_valid']=='是'))].agg({'CGI':'count'})[0]
cmcc_srtong_fa = data.loc[((data['un_valid']=='是')&(data['te_valid']=='是'))].agg({'CGI':'count'})[0]
cmcc_srtong_v = cmcc_srtong_son/cmcc_srtong_fa

In [ ]:
weak_count = data.loc[((data['cmcc'] <0.9)|((data['te_hd']=='黑点')|(data['te_hd']=='黑点')))].agg({'CGI':'count'})[0]

In [ ]:
dict_data = {'MR软采小区数': data_row,
             '采集率': '',
             '全网MR覆盖率': cmcc_fgl,
            '核心城区MR覆盖率': cmcc_atu_fgl,
            'MR弱覆盖小区数': data_weak,
            'MR弱覆盖小区占比': data_weak/data_row,
            'MR劣于竞对小区数': hd_use,
            'MR劣于竞对小区占比': '',
            'MR强于竞对小区数': cmcc_srtong_son,
            'MR强于竞对小区占比': cmcc_srtong_v,
            'MR问题小区数': problem_cell,
            'MR问题小区占比': problem_cell/data_row}

In [ ]:
result = pd.DataFrame.from_dict(dict_data, orient='index', columns=['values'])

In [ ]:
data.to_csv('c:/Users/Administrator/desktop/aaa.csv',encoding='gbk')

In [ ]:
result.to_clipboard()

In [ ]:
result